In [33]:
from binance.client import Client
from binance import ThreadedWebsocketManager
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
api_key = 'haWzm6Ujj0SdpaCeCBiyyAvTvS78odvplgl4W9t29CVgSq5LrAHpt940JYQCt5TD'
secret_key = 'SgoyFqlJ6SvCYSXPFsU1ehPzQHhsfFV3Pa5scLF6y0stOO0aYXpfig6Fy4N0ePWX'
client = Client(api_key=api_key, api_secret=secret_key, tld='com')
account = client.get_account()

In [42]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
        
    def define_strategy(self): # "strategy-specific"
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
        

...

In [43]:
symbol = 'BTCUSDT'
bar_length = '1m'
return_thresh = 0
volume_thresh = [-3,3]

In [44]:
trader = LongOnlyTrader(symbol=symbol, bar_length=bar_length, return_thresh=return_thresh, volume_thresh=volume_thresh)
trader

In [48]:
trader.start_trading(1/24)

Task was destroyed but it is pending!
task: <Task pending coro=<ReconnectingWebsocket._read_loop() done, defined at c:\Users\AndreyIchshenko\AppData\Local\Programs\Python\Python37\lib\site-packages\binance\streams.py:122> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x000002218153CD38>()]>>


...

................................................

In [50]:
trader.twm.stop()

In [51]:
trader.prepared_data


,Close,Volume,returns,vol_ch,position
Date,,,,,
2022-09-12 19:26:00,22398.15,126.44737,NaN,NaN,1
2022-09-12 19:27:00,22392.16,166.09754,-0.000267,0.272749,1
2022-09-12 19:28:00,22399.40,152.30928,0.000323,-0.086662,0
2022-09-12 19:29:00,22395.97,131.04433,-0.000153,-0.150378,1
2022-09-12 19:30:00,22390.40,186.23691,-0.000249,0.351484,1
2022-09-12 19:31:00,22401.60,183.37571,0.000500,-0.015482,0
2022-09-12 19:32:00,22414.90,178.70016,0.000594,-0.025828,0
2022-09-12 19:33:00,22391.23,173.72375,-0.001057,-0.028243,1
2022-09-12 19:34:00,22390.25,243.08408,-0.000044,0.335941,1


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................